<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Simple Decision Prompts</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundamental building blocks for eliciting effective responses from AI models. This module demonstrates how to use common prompt engineering techniques within Semantic Kernel. If you’ve used ChatGPT or Microsoft Copilot, you’re already familiar with prompting: given an instruction, a language model predicts the most likely and relevant response. With Semantic Kernel, you can build applications, services, and APIs that execute these prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal is to maximize the quality and clarity of the AI’s output, often by using specific wording, added context, or concrete examples within the prompt.

By combining Decision Intelligence with Prompt Engineering, you can create “Generative AI Decision Intelligence.” This approach leverages GenAI models to reason through complex tasks, gather intelligence, recommend decisions, and communicate results more effectively.

-----
### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [3]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connection info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6

In [5]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.6.1"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.6.1

Using Azure OpenAI Service


-----
### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [6]:
// A Decision Intelligence prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

General Formatting Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
var response = simplePromptResponse.GetValue<string>(); 

// Display the response from the Semantic Kernel as Markdown format
response.DisplayAs("text/markdown");

### 1. SWOT Analysis  
SWOT (Strengths, Weaknesses, Opportunities, Threats) helps individuals or groups systematically evaluate internal and external factors affecting their options. By listing strengths and weaknesses (internal) alongside opportunities and threats (external), decision-makers can weigh pros and cons, anticipate risks, and identify strategic advantages, resulting in more balanced and comprehensive choices.

### 2. Cost-Benefit Analysis (CBA)  
CBA involves listing and quantifying all expected costs and benefits of possible options. This framework drives objective reasoning by allowing decision-makers to compare alternatives based on their overall net value. It is especially valuable for financial or resource allocation decisions, as it clarifies which actions deliver the greatest benefit for the least cost.

### 3. The Six Thinking Hats  
Developed by Edward de Bono, this framework encourages considering a decision from six distinct perspectives (hats): facts, emotions, negative judgment, positive judgment, creativity, and process. This structured approach ensures multifaceted analysis—minimizing bias, uncovering hidden risks or opportunities, and promoting creative solutions by deliberately examining all sides.

### 4. The OODA Loop  
OODA stands for Observe, Orient, Decide, Act. Originally used in military strategy, it applies to various fast-moving or competitive contexts. By iterating through these steps, decision-makers remain adaptive: they gather information, interpret it against their understanding, decide on a course of action, and execute—then loop back to observe outcomes. This framework sharpens situational awareness and responsiveness.

### 5. Decision Matrix (Weighted Scoring Model)  
A decision matrix allows stakeholders to assess multiple options against set criteria, assigning weights based on importance. Each option is scored for how well it meets each criterion, and weighted totals are calculated. This supports transparent, rational analysis—favoring choices that best align with priorities and enabling clear comparison, even when decisions are complex and multidimensional.

-----
### Step 3 - Execute a Decision Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> 📝 **Note:** An average human can read between 25-40 Tokens / second. Therefore, while streaming certainly helps with providing AI output to the user, it begins to lose its effectiveness at large token velocity. Furthermore, certain intermediate and governance (Responsible AI) outputs can become more complex to integrat with streaming.

In [7]:
// Decision Prompt using Streaming output chunks 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

### 1. SWOT Analysis

**Description:**  
SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This framework enables decision-makers to systematically evaluate internal (strengths, weaknesses) and external (opportunities, threats) factors related to a problem or opportunity.  

**How it supports better analysis:**  
By clearly mapping out these four quadrants, individuals or teams gain a holistic view of a situation, identify areas to leverage or improve, and anticipate potential challenges. This balance of internal and external perspectives ensures more robust and defensible decisions.

---

### 2. Cost-Benefit Analysis (CBA)

**Description:**  
CBA involves listing and comparing the costs and benefits of different courses of action, typically quantifying them in monetary terms where possible.  

**How it supports better analysis:**  
By translating qualitative and quantitative factors into comparable units, CBA helps to clarify trade-offs, prioritize options, and make t

-----
### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can output Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting output. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [8]:
// A new decision prompt to help with describing decision-making frameworks using Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat);
var response = simplePromptResponse.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework              | Description                                                                                                                                      | How It Supports Better Analysis and Reasoning                                                                                       |
|------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis          | Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision, project, or organization.                                      | Encourages comprehensive situational understanding, balancing internal and external factors for more informed strategies.            |
| Cost-Benefit Analysis  | Compares the costs and benefits of alternative choices to determine their economic viability.                                                     | Provides a quantitative basis for evaluating options, clarifying trade-offs, and supporting rational, value-driven decisions.        |
| The Five Whys          | Repeatedly asks "Why?" to drill down to the root cause of a problem or decision.                                                                 | Reveals underlying causes, reducing superficial reasoning and promoting solutions that address core issues for better outcomes.      |
| Decision Matrix        | Lists options against weighted criteria to objectively score and rank alternatives.                                                               | Structures evaluation, reduces bias, and enables side-by-side comparison, resulting in more objective and transparent decisions.     |
| OODA Loop              | Stands for Observe, Orient, Decide, Act—a cyclical process for decision-making in dynamic environments.                                           | Promotes situational awareness, rapid assessment, adaptation, and continuous learning for agile, evidence-based decision-making.     |

-----
### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Temperature or other settings.

> **📝 Note:** The supported prompt settings are dependent on the API plus the specific model version. For example, an AI model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available. Types of model execution (general versus reasoning) will also have different execution setting parameters. 

In [9]:
// Create a new OpenAI Prompt Execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Create a new KernelArguments object with the OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseWithOpenAI = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat, kernelArguments);
var response = promptResponseWithOpenAI.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework                | Description                                                                                      | How It Enhances Decision Quality                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------|
| SWOT Analysis            | Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.    | Encourages comprehensive internal and external analysis, clarifying risks and advantages. |
| Cost-Benefit Analysis    | Compares the costs and benefits of different options quantitatively or qualitatively.             | Supports rational choices by weighing pros and cons, aiding resource allocation.          |
| Decision Matrix          | Ranks options against weighted criteria to identify the most suitable choice.                     | Provides structured, objective comparison, reducing bias and clarifying priorities.       |
| Six Thinking Hats        | Uses six distinct perspectives (hats) to examine a problem from multiple angles.                  | Promotes balanced, creative, and critical thinking, reducing blind spots.                |
| OODA Loop                | Cycles through Observe, Orient, Decide, and Act for rapid, adaptive decision-making.              | Enhances responsiveness and iterative improvement, especially in dynamic environments.    |

-----
### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decision Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific (i.e. decision intelligence)
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [25]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);


var promptResponseWithTemplate = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt, kernelArguments);
var response = promptResponseWithTemplate.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

### 1. Pros and Cons List

**How it works:**  
List the positive (pros) and negative (cons) aspects of each option. Compare the lists to see which option has more advantages or fewer drawbacks.

**Why it’s useful:**  
- Simple and intuitive  
- Helps clarify trade-offs  
- Can be used for almost any decision, big or small

**Example:**  
Should I cook dinner or order takeout? List pros and cons for each.

  

### 2. Eisenhower Matrix (Urgent-Important Matrix)

**How it works:**  
Categorize tasks or decisions into four quadrants:  
- Urgent and Important  
- Important but Not Urgent  
- Urgent but Not Important  
- Neither Urgent nor Important

**Why it’s useful:**  
- Helps prioritize actions  
- Reduces overwhelm by focusing on what truly matters

**Example:**  
Deciding which tasks to do first in a busy day.

  

### 3. 2x2 Decision Grid

**How it works:**  
Plot options on a grid with two key criteria (e.g., cost vs. benefit, effort vs. impact). This visualizes which options offer the best trade-off.

**Why it’s useful:**  
- Quick visual comparison  
- Forces you to focus on what matters most

**Example:**  
Choosing between different weekend activities based on fun and cost.

  

### 4. The Five Whys

**How it works:**  
Ask "Why?" five times (or as many as needed) to get to the root cause or underlying motivation for a decision.

**Why it’s useful:**  
- Clarifies the real problem or need  
- Prevents superficial decision-making

**Example:**  
Why am I considering changing jobs? (Keep asking why to uncover true motivations.)

  

### 5. OODA Loop (Observe, Orient, Decide, Act)

**How it works:**  
- Observe: Gather information  
- Orient: Analyze and interpret  
- Decide: Choose an action  
- Act: Implement the decision  
Repeat as needed.

**Why it’s useful:**  
- Encourages quick, iterative decision-making  
- Adaptable to changing situations

**Example:**  
Deciding how to respond to an unexpected event during your day.

  

### Summary Table

| Framework           | Best For                       | Key Benefit                |
|---------------------|-------------------------------|----------------------------|
| Pros and Cons List  | Everyday choices              | Clarity and simplicity     |
| Eisenhower Matrix   | Task prioritization           | Focus on importance        |
| 2x2 Decision Grid   | Comparing options             | Visual decision support    |
| Five Whys           | Understanding motivations     | Root cause analysis        |
| OODA Loop           | Dynamic situations            | Fast, adaptive decisions   |

These frameworks are easy to learn, flexible, and can be applied to a wide range of daily decisions.

-----
### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full **Decision Intelligence** framework will not be used, rather a simple request for Artificial Intelligence to recommend a path forward (recommendation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [26]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed Decision Scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommendation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, // Increase the max tokens to allow for a more detailed response
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseScenario = await semanticKernel.InvokePromptAsync(scenarioDecisionPromptTemplate, kernelArguments);
var response = promptResponseScenario.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

### Recommendation Table for Alex: Community College First, Then Transfer

| Decision Factor                | Community College First (Recommended)                                  | Four-Year University Direct Entry                         | Reasoning for Recommendation                                                                                   |
|-------------------------------|-----------------------------------------------------------------------|----------------------------------------------------------|----------------------------------------------------------------------------------------------------------------|
| Financial Considerations      | Substantial cost savings; lower tuition and living expenses; less debt | Much higher cost; potential for significant student loans | Cost is a medium concern, but saving $90,000+ over two years reduces long-term debt and financial pressure.    |
| Career and Major Uncertainty  | Flexibility to explore majors at low cost; easier to change direction  | Risk of extra time/cost if switching majors               | Community college allows academic exploration without high financial risk, ideal given Alex’s uncertainty.      |
| Academic Consistency & Networking | Delayed access to university networking, but can join later; smaller classes for first two years | Immediate access to university resources and networks     | While early networking is valuable, Alex can still build connections after transferring, and smaller classes may offer more support during exploration. |
| Future Impact                 | Lower debt, more flexibility; may require adjustment when transferring | Stronger early network, but higher debt and risk if major changes | Community college reduces financial risk and allows Alex to clarify her goals before committing to a major investment. |

### Summary

Starting at a community college is recommended for Alex. This path provides significant financial savings, reduces the risk associated with major uncertainty, and offers flexibility to explore academic interests. Although it may delay some networking opportunities, Alex can still build a strong professional network after transferring to a four-year university. This approach minimizes long-term debt and allows for a more informed decision about her major and career path, supporting both her immediate academic needs and her future financial stability.